# Status Codes

> Create a treemap for visualizing status code on two levels: the status category (100, 200, 300, 400, 500) and the status codes' actual values (200, 301, 404, etc.)

## Input: 

A list of of status codes.

## Output:

A `plotly.graph_objects.Figure` object. A treemap with two levels showing the distribution/proportions. 

In [ ]:
# | default_exp status_codes

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
from http.client import responses

import pandas as pd
import plotly.express as px

In [ ]:
# | export
_texttemplate = "<b>%{label} </b><br><br>Status code: %{label} %{customdata[0]}<br>Count: %{value:,}<br>%{percentParent:.1%} of %{parent}<br>"  # %{percentRoot:.1%} of %{root}'

In [ ]:
# | export
def status_codes(
    status_list,
    height=600,
    width=None,
    template="none",
    title="Status Codes",
    subtitle=None,
):
    """Create a treemap to visualize a list of status codes.

    Parameters
    ----------

    status_list : list, tuple, pandas.Series
        A collection of HTTP status codes
    height: integer
        The desired height of the figure in pixels
    width: integer
        The desired width of the figure in pixels
    template : str
        Name of template to use for the chart. Available themes:
            ggplot2, seaborn, simple_white, plotly, plotly_white, plotly_dark,
            presentation, xgridoff, ygridoff, gridon, none
    title : str
        The title of the figure. You can use/include the following HTML tags in
        the title: `<a>`, `<b>`, `<br>`, `<i>`, `<sub>`, `<sup>`
    subtitle : str
        The subtitle of the figure, by default 70% of the font size of the title

    Returns
    -------
    status_codes_figure : plotly.graph_objects.Figure
    """
    status_counts = pd.Series(status_list).value_counts().reset_index()
    status_counts.columns = ["status", "count"]
    status_counts = status_counts.assign(
        status_cat=lambda df: df["status"].astype(int).round(-2)
    ).assign(status_desc=lambda df: [responses[int(code)] for code in df["status"]])
    status_fig = px.treemap(
        status_counts,
        maxdepth=2,
        path=[px.Constant("Status codes"), "status_cat", "status"],
        hover_name="status_cat",
        height=height,
        width=width,
        custom_data=["status_desc"],
        values="count",
        title=title,
        subtitle=subtitle,
        template=template,
    )
    status_fig.data[0].marker.line.width = 0.01
    status_fig.data[0].marker.pad = dict.fromkeys("lrbt", 0)
    status_fig.data[0]["texttemplate"] = _texttemplate
    status_fig.data[0]["hovertemplate"] = _texttemplate
    return status_fig

### Visualize the distribution of a list of status codes

In [ ]:
codes = pd.read_csv("data/status_codes.csv")
codes

,status
0,200
1,200
2,200
3,200
4,200
...,...
414398,200
414399,200
414400,200
414401,200


In [ ]:
status_codes(codes["status"])

## Change the template with the `template` parameter

In [ ]:
status_codes(codes["status"], template="seaborn")

In [ ]:
status_codes(codes["status"], template="plotly_dark")

## Change the dimensions of the charts with `width` and `height`

In [ ]:
status_codes(codes["status"], width=700, height=700, subtitle="example.com")

## Modify the title of the chart, and make use of various HTML tags `<a>`, `<b>`, `<br>`, `<i>`, `<sub>`, `<sup>`

In [ ]:
website = "MyWebsite.com"
title = f'Error Codes distribution <b>400</b> and <b>500</b><br>clickable: <i><a href="https://mywebsite.com">{website}</i> – April 3, 2023'

status_codes(
    codes.query("status >= 400")["status"],
    title=title,
    height=700,
    width=1300,
    template="ggplot2",
)

### Try downloading, interacting with, and emailing this chart. [File link](data/status_code_chart.html)

In [ ]:
# | hide
# | echo: false
# | execute: false
status_codes(codes["status"], theme="plotly_dark").write_image(
    "data/images/status_codes_chart.png"
)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()